# Analysis

There are two classes for analysis: Alternative which provides methods for a [Cost Benefit Analysis](https://en.wikipedia.org/wiki/Cost–benefit_analysis) and Decision which provides methods for a [Multiple Criteria Analysis](https://en.wikipedia.org/wiki/Multiple-criteria_decision_analysis). The Timeline class generates a [Gantt Chart](https://en.wikipedia.org/wiki/Gantt_chart).

## Alternative

In [ ]:
from datetime import datetime, timedelta
from dpd.analysis import Activity, Alternative, Currency

USD = Currency("USD", 2020, 0.07).discount()

a1 = Alternative(name="Alternative One")
planning = Activity(
    "Planning", datetime(2005, 1, 1), datetime(2009, 12, 31), 100 * USD, 0 * USD
)
construction = Activity(
    "Construction", datetime(2010, 1, 1), datetime(2014, 12, 31), 100 * USD, 0 * USD
)
operations = Activity(
    "Operations",
    construction.End,
    construction.End + timedelta(days=365 * 20 + 4),
    0 * USD,
    400 * USD,
)

for activity in [planning, construction, operations]:
    a1.add_activity(activity)

a1

In [ ]:
a1.benefit_cost_ratio

In [ ]:
a1.period_range_pivot().head()

In [ ]:
from matplotlib import pyplot as plt

fig, ax = plt.subplots()

a1.cash_flow_diagram(ax=ax)
ax.get_legend().remove()

## Timeline

In [ ]:
from dpd.analysis import Timeline

t = Timeline()
for activity in [planning, construction, operations]:
    t.add_activity(activity)

fig, ax = plt.subplots()
t.plot_gantt(ax=ax)
plt.show()

In [ ]:
fig, ax = plt.subplots()
a1.timeline.plot_gantt(ax=ax)
plt.show()

## Decision

In [ ]:
from datetime import datetime
from random import randint

from astropy import units

from dpd.analysis import Alternative, Currency, Decision
from dpd.analysis.criteria import AttributeCriterion, CostCriterion, Criterion

USD = Currency("USD", 2020, 0.07).discount()
decision = Decision()

no_build_alternative = Alternative("No Build Alternative")
no_build_alternative.some_cost = 0 * USD
alternative_one = Alternative("Alternative 1")
alternative_one.some_cost = 100 * USD
alternative_two = Alternative("Alternative 2")
alternative_two.some_cost = 200 * USD


@property
def travel_time(self):
    return randint(100, 200) * units.second


setattr(Alternative, "travel_time", travel_time)

decision.add_criterion(AttributeCriterion("Travel Time", weight=1 / units.second))
decision.add_criterion(
    Criterion("Distance", lambda x: 200 * units.meter, weight=1 / units.meter)
)
decision.add_criterion(AttributeCriterion("Some Cost", weight=1 / USD))

for alternative in [no_build_alternative, alternative_one, alternative_two]:
    decision.add_alternative(alternative)

decision

In [ ]:
decision.plot(kind="bar")

In [ ]:
decision.multiple_criteria_decision_analysis().plot(kind="bar")

## Values

In [ ]:
from dpd.analysis.values import (
    Value_of_Reduced_Fatalities_and_Injuries,
    Value_of_Travel_Time_Savings,
)

In [ ]:
Value_of_Reduced_Fatalities_and_Injuries

In [ ]:
Value_of_Travel_Time_Savings

In [ ]:
from astropy.units import day, minute, year
from dpd.analysis.units import person, usd2020

In [ ]:
(
    5
    * minute
    * 40000
    * person
    / day
    * Value_of_Travel_Time_Savings[
        "Walking Cycling, Waiting, Standing, and Transfer Time"
    ]
).decompose().to(usd2020 / year)